In [73]:
import pandas as pd

df = pd.read_csv('tools-for-thought.csv')
sample = df.sample(100)


In [58]:
# mark standalone, reply, rt
sample['rt'] = sample["referenced_tweets.retweeted.id"].isna().apply(lambda x: not x)
sample['reply'] = sample["referenced_tweets.replied_to.id"].isna().apply(lambda x: not x)
sample['quote'] = sample["referenced_tweets.quoted.id"].isna().apply(lambda x: not x)
sample['standalone'] = df[["referenced_tweets.replied_to.id",	"referenced_tweets.retweeted.id", "referenced_tweets.quoted.id"]].isna().sum(axis=1).apply(lambda x: True if x == 3 else False)

# rename author.name to name
sample = sample.rename(columns={'author.name': 'name', 'author.username': 'username'})

# find all columns starting with 'public metrics'
names = [col for col in df.columns if col.startswith('public_metrics')]
# remove 
new_names = [col.split('.')[1] for col in names]
sample = sample.rename(columns=dict(zip(names, new_names)))

In [59]:
# Convert to desired schema

sample['id'] = sample['id'].astype(str)
sample['author'] = sample[['name', 'username']].to_dict(orient='records')
sample['html'] = sample['text'].astype(str)
sample['public_metrics'] = sample[new_names].to_dict(orient='records')

sample = sample[['id', 'author', 'html', 'created_at', 'public_metrics', 'standalone', 'rt', 'reply', 'quote'] + ['geo.country', 'entities.annotations', 'context_annotations']]

sample.to_json('sample.json', orient='records')